In [22]:
!pip install dart-fss -qq
"998a788dbaa03a8a9a20d9113c6515c538994d89"

In [31]:
import pandas as pd
import requests
from io import BytesIO

def get_konex_stocks():
    """
    한국거래소(KRX)에서 KONEX 시장에 상장된 기업들의 종목코드를 가져오는 함수
    """
    # 한국거래소에서 KONEX 시장 종목 정보를 제공하는 URL
    url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    
    # HTTP 요청에 필요한 form data (KONEX 시장 코드 적용)
    params = {
        "code": "ALL",  # KONEX 전체 종목
        "market_gubun": "konexMkt"  # KONEX 시장 지정
    }
    
    # HTTP POST 요청
    response = requests.post(url, data=params)
    
    if response.status_code == 200:
        # 응답 데이터를 CSV로 변환
        df = pd.read_csv(BytesIO(response.content), encoding="EUC-KR")
        
        # 종목코드만 추출
        konex_codes = df[['단축코드', '한글 종목명']]
        konex_codes.columns = ['종목코드', '종목명']
        
        return konex_codes
    else:
        print(f"데이터 요청 실패: {response.status_code}")
        return None

# 실행 예시
if __name__ == "__main__":
    konex_list = get_konex_stocks()
    
    if konex_list is not None:
        print(f"KONEX 상장 기업 수: {len(konex_list)}")
        print(konex_list.head())  # 상위 5개 데이터 출력


데이터 요청 실패: 403


In [32]:
import pandas as pd
import requests
import dart_fss as dart
from io import BytesIO

# DART API KEY 설정 (본인의 API 키 입력)
API_KEY = "998a788dbaa03a8a9a20d9113c6515c538994d89"
dart.set_api_key(api_key=API_KEY)

def get_kosdaq_stocks():
    """
    한국거래소(KRX)에서 코스닥 시장 상장 기업들의 종목코드를 가져오는 함수
    """
    # KRX에서 코스닥 상장기업 목록 제공 URL
    url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    
    # HTTP 요청에 필요한 form data (코스닥 시장 지정)
    params = {
        "code": "ALL",  # 전체 종목
        "market_gubun": "kosdaqMkt"  # 코스닥 시장
    }

    # HTTP 요청
    response = requests.post(url, data=params)

    if response.status_code == 200:
        # 응답 데이터를 DataFrame으로 변환
        df = pd.read_csv(BytesIO(response.content), encoding="EUC-KR")
        
        # 필요한 컬럼만 추출 (종목코드, 종목명)
        kosdaq_codes = df[['단축코드', '한글 종목명']]
        kosdaq_codes.columns = ['종목코드', '기업명']
        
        return kosdaq_codes
    else:
        print(f"데이터 요청 실패: {response.status_code}")
        return None

def get_financial_statements(corp_code, year=2023):
    """
    특정 기업의 재무제표를 조회하여 DataFrame으로 변환하는 함수
    :param corp_code: 기업 코드 (DART 기업 코드)
    :param year: 조회할 연도 (기본값: 2023)
    """
    try:
        # 해당 연도의 연결재무제표 가져오기
        fs = dart.fs.extract(corp_code=corp_code, report_tp='annual', fs_tp='consolidated', bgn_de=f"{year}0101")
        
        # 재무 데이터 DataFrame 변환
        if fs:
            df = fs.to_DataFrame()
            df['기업코드'] = corp_code  # 기업 코드 추가
            return df
        else:
            return None
    except Exception as e:
        print(f"{corp_code} 재무제표 조회 오류: {e}")
        return None

# 실행 예제
if __name__ == "__main__":
    # 1. KOSDAQ 상장 기업 리스트 가져오기
    kosdaq_list = get_kosdaq_stocks()

    if kosdaq_list is not None:
        print(f"KOSDAQ 상장 기업 수: {len(kosdaq_list)}")

        # DART API에서 기업 코드 찾기
        corp_list = dart.get_corp_list()

        # 2. 각 기업의 재무제표 가져오기
        all_financial_data = []

        for idx, row in kosdaq_list.iterrows():
            stock_code = row['종목코드']
            company_name = row['기업명']

            # DART API에서 해당 종목의 기업 코드 찾기
            corp_info = corp_list.find_by_stock_code(stock_code)

            if corp_info:
                corp_code = corp_info.corp_code
                print(f"재무제표 조회 중: {company_name} ({corp_code})")
                
                # 기업 재무제표 가져오기
                financial_df = get_financial_statements(corp_code, year=2023)

                if financial_df is not None:
                    all_financial_data.append(financial_df)

        # 3. 전체 재무 데이터 DataFrame으로 변환
        if all_financial_data:
            final_df = pd.concat(all_financial_data, ignore_index=True)
            print(final_df.head())  # 상위 5개 출력
        else:
            print("재무 데이터가 없습니다.")


데이터 요청 실패: 403


In [39]:
import pandas as pd
import requests
from io import BytesIO

def get_kosdaq_stocks():
    """
    한국거래소(KRX)에서 코스닥 시장 상장 기업들의 종목코드를 가져오는 함수
    """
    # KRX에서 코스닥 상장기업 목록 제공 URL
    url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    
    # HTTP 요청에 필요한 form data (코스닥 시장 지정)
    params = {
        "code": "ALL",  # 전체 종목
        "market_gubun": "kosdaqMkt"  # 코스닥 시장
    }
    
    # User-Agent 헤더 추가 (브라우저처럼 보이도록)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # 세션을 사용하여 요청
    with requests.Session() as session:
        session.headers.update(headers)
        response = session.post(url, data=params)

    if response.status_code == 200:
        # 응답 데이터를 DataFrame으로 변환
        df = pd.read_csv(BytesIO(response.content), encoding="EUC-KR")
        
        # 필요한 컬럼만 추출 (종목코드, 종목명)
        kosdaq_codes = df[['단축코드', '한글 종목명']]
        kosdaq_codes.columns = ['종목코드', '기업명']
        
        return kosdaq_codes
    else:
        print(f"데이터 요청 실패: {response.status_code}")
        return None

# 실행 예시
if __name__ == "__main__":
    kosdaq_list = get_kosdaq_stocks()
    if kosdaq_list is not None:
        print(f"KOSDAQ 상장 기업 수: {len(kosdaq_list)}")
        print(kosdaq_list.head())  # 상위 5개 출력


데이터 요청 실패: 403
